In [1]:
# ===============================
# 🔹 1. Install Fairness Libraries
# ===============================
!pip install aif360 pandas numpy scikit-learn seaborn matplotlib --quiet

# If Colab throws IBM AIF360 error, run these two:
!pip install aif360[all] --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.7/259.7 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 56.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.8/515.8 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [2]:
# ===============================
# 🔹 2. Import Libraries
# ===============================
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.algorithms.preprocessing import Reweighing


/usr/local/lib/python3.12/dist-packages/inFairness/utils/ndcg.py:37: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  vect_normalized_discounted_cumulative_gain = vmap(
/usr/local/lib/python3.12/dist-packages/inFairness/utils/ndcg.py:48: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  monte_carlo_vect_ndcg = vmap(vect_normalized_discounted

In [3]:
# ===============================
# 🔹 3. Load the Loan Dataset
# ===============================
df = pd.read_csv("loan.csv")   # Upload in Colab → use exact name

print("Shape:", df.shape)
df.head()


Shape: (614, 13)


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [4]:
# ===============================
# 🔹 4. Basic Preprocessing
# ===============================

# Example categorical encodings (unique smart approach)
df = df.drop_duplicates()

# Automatically encode label-type columns
enc = LabelEncoder()
for col in df.select_dtypes(include=['object']).columns:
    df[col] = enc.fit_transform(df[col].astype(str))

df = df.dropna()

df.head()


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,1,1,1,1,0,0,4583,1508.0,128.0,360.0,1.0,0,0
2,2,1,1,0,0,1,3000,0.0,66.0,360.0,1.0,2,1
3,3,1,1,0,1,0,2583,2358.0,120.0,360.0,1.0,2,1
4,4,1,0,0,0,0,6000,0.0,141.0,360.0,1.0,2,1
5,5,1,1,2,0,1,5417,4196.0,267.0,360.0,1.0,2,1


In [5]:
# ===============================
# 🔹 5. Feature Selection
# ===============================
X = df.drop("Loan_Status", axis=1)   # Change column name if different
y = df["Loan_Status"]

sensitive = "Gender"     # Protected attribute
privileged_group = [{sensitive: 1}]   # Example: Male = 1
unprivileged_group = [{sensitive: 0}] # Female = 0

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


In [7]:
# ===============================
# FIXED STEP 6: Create AIF360 Dataset Safely
# ===============================

# Convert scaled features back into a dataframe
X_train_df = pd.DataFrame(X_train, columns=X.columns)
X_test_df = pd.DataFrame(X_test, columns=X.columns)

# Add label column back
X_train_df['Loan_Status'] = y_train.values
X_test_df['Loan_Status'] = y_test.values

# ---- IMPORTANT ----
# Make sure your sensitive attribute exists here
print("Columns in training dataset:", X_train_df.columns)

# Convert AIF360 datasets
train_bds = BinaryLabelDataset(
    df=X_train_df,
    label_names=["Loan_Status"],
    protected_attribute_names=[sensitive],
    favorable_label=1,
    unfavorable_label=0
)

test_bds = BinaryLabelDataset(
    df=X_test_df,
    label_names=["Loan_Status"],
    protected_attribute_names=[sensitive],
    favorable_label=1,
    unfavorable_label=0
)


Columns in training dataset: Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')


In [8]:
# ===============================
# STEP 7: Train Base Classifier
# ===============================

model = LogisticRegression(max_iter=500)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.7969924812030075

Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.39      0.54        41
           1       0.78      0.98      0.87        92

    accuracy                           0.80       133
   macro avg       0.84      0.68      0.71       133
weighted avg       0.82      0.80      0.77       133



In [9]:
# ===============================
# STEP 8: Dataset Bias Metrics
# ===============================

metric_train = BinaryLabelDatasetMetric(
    train_bds,
    privileged_groups=[{sensitive: 1}],
    unprivileged_groups=[{sensitive: 0}]
)

print("🔹 Mean Difference (Demographic Parity):", metric_train.mean_difference())
print("🔹 Disparate Impact:", metric_train.disparate_impact())


🔹 Mean Difference (Demographic Parity): nan
🔹 Disparate Impact: nan


/usr/local/lib/python3.12/dist-packages/aif360/metrics/binary_label_dataset_metric.py:105: RuntimeWarning: invalid value encountered in scalar divide
  return (self.num_positives(privileged=privileged)


In [10]:
# ===============================
# STEP 9: Classification Fairness Metrics
# ===============================

test_pred_bds = test_bds.copy()
test_pred_bds.labels = y_pred.reshape(-1, 1)

metric_classifier = ClassificationMetric(
    test_bds,
    test_pred_bds,
    privileged_groups=[{sensitive: 1}],
    unprivileged_groups=[{sensitive: 0}]
)

print("🔸 Equal Opportunity Difference:", metric_classifier.equal_opportunity_difference())
print("🔸 Average Odds Difference:", metric_classifier.average_odds_difference())


🔸 Equal Opportunity Difference: nan
🔸 Average Odds Difference: nan


/usr/local/lib/python3.12/dist-packages/aif360/metrics/classification_metric.py:278: RuntimeWarning: invalid value encountered in scalar divide
  TPR=TP / P, TNR=TN / N, FPR=FP / N, FNR=FN / P,
/usr/local/lib/python3.12/dist-packages/aif360/metrics/classification_metric.py:279: RuntimeWarning: invalid value encountered in scalar divide
  GTPR=GTP / P, GTNR=GTN / N, GFPR=GFP / N, GFNR=GFN / P,


In [11]:
# ===============================
# STEP 10: Fairness Mitigation
# ===============================

RW = Reweighing(
    privileged_groups=[{sensitive: 1}],
    unprivileged_groups=[{sensitive: 0}]
)

train_transformed = RW.fit_transform(train_bds)

# Train fair classifier
model_fair = LogisticRegression(max_iter=500)
model_fair.fit(X_train, y_train, sample_weight=train_transformed.instance_weights)

y_pred_fair = model_fair.predict(X_test)

print("Fair Model Accuracy:", accuracy_score(y_test, y_pred_fair))


Fair Model Accuracy: 0.7969924812030075


/usr/local/lib/python3.12/dist-packages/aif360/algorithms/preprocessing/reweighing.py:66: RuntimeWarning: invalid value encountered in scalar divide
  self.w_p_fav = n_fav*n_p / (n*n_p_fav)
/usr/local/lib/python3.12/dist-packages/aif360/algorithms/preprocessing/reweighing.py:67: RuntimeWarning: invalid value encountered in scalar divide
  self.w_p_unfav = n_unfav*n_p / (n*n_p_unfav)
/usr/local/lib/python3.12/dist-packages/aif360/algorithms/preprocessing/reweighing.py:68: RuntimeWarning: invalid value encountered in scalar divide
  self.w_up_fav = n_fav*n_up / (n*n_up_fav)
/usr/local/lib/python3.12/dist-packages/aif360/algorithms/preprocessing/reweighing.py:69: RuntimeWarning: invalid value encountered in scalar divide
  self.w_up_unfav = n_unfav*n_up / (n*n_up_unfav)


In [12]:
# ===============================
# STEP 11: Fairness After Mitigation
# ===============================

test_pred_fair_bds = test_bds.copy()
test_pred_fair_bds.labels = y_pred_fair.reshape(-1, 1)

metric_fair = ClassificationMetric(
    test_bds,
    test_pred_fair_bds,
    privileged_groups=[{sensitive: 1}],
    unprivileged_groups=[{sensitive: 0}]
)

print("✨ After Mitigation Results ✨")
print("Equal Opportunity Difference:", metric_fair.equal_opportunity_difference())
print("Average Odds Difference:", metric_fair.average_odds_difference())
print("Disparate Impact:", metric_fair.disparate_impact())


✨ After Mitigation Results ✨
Equal Opportunity Difference: nan
Average Odds Difference: nan
Disparate Impact: nan


/usr/local/lib/python3.12/dist-packages/aif360/metrics/classification_metric.py:278: RuntimeWarning: invalid value encountered in scalar divide
  TPR=TP / P, TNR=TN / N, FPR=FP / N, FNR=FN / P,
/usr/local/lib/python3.12/dist-packages/aif360/metrics/classification_metric.py:279: RuntimeWarning: invalid value encountered in scalar divide
  GTPR=GTP / P, GTNR=GTN / N, GFPR=GFP / N, GFNR=GFN / P,
/usr/local/lib/python3.12/dist-packages/aif360/metrics/classification_metric.py:673: RuntimeWarning: invalid value encountered in scalar divide
  return (self.num_pred_positives(privileged=privileged)


In [13]:
print(df.columns)


Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')


In [18]:
sensitive = "Gender"
y = df["Loan_Status"]


In [19]:
# ==========================================
# STEP A: Loan approval % for each gender
# ==========================================

gender_col = sensitive   # e.g., "Gender"
label_col = "Loan_Status"

approval_rates = df.groupby(gender_col)[label_col].mean() * 100

print("Loan Approval Percentage by Gender:\n")
print(approval_rates)

# Pretty output
for g, v in approval_rates.items():
    print(f"Gender {g}: {v:.2f}% approved")


Loan Approval Percentage by Gender:

Gender
0    64.210526
1    70.616114
2    58.333333
Name: Loan_Status, dtype: float64
Gender 0: 64.21% approved
Gender 1: 70.62% approved
Gender 2: 58.33% approved


In [20]:
# ==========================================
# STEP B: Count approved vs not approved
# ==========================================

summary = df.groupby(gender_col)[label_col].value_counts(normalize=True).unstack() * 100
summary.columns = ["Rejected %", "Approved %"]
print(summary)


        Rejected %  Approved %
Gender                        
0        35.789474   64.210526
1        29.383886   70.616114
2        41.666667   58.333333
